In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable

In [2]:
torch.__version__

'0.1.12_2'

# Load Datasets

In [3]:
mnist_data_path = './data/pytorch_data'
transform = torchvision.transforms.ToTensor() # transform PIL image to Tensor
mnist_train = torchvision.datasets.MNIST(mnist_data_path, train=True, transform=transform,
                                         target_transform=None, download=True)
mnist_test  = torchvision.datasets.MNIST(mnist_data_path, train=False, transform=transform,
                                         target_transform=None,download=True)
print("train_len {}, test_len {}".format(len(mnist_train), len(mnist_test)))

train_len 60000, test_len 10000


In [4]:
# See some examples
sample = mnist_train[0]
print("Sample class: {}".format(sample[1]))
print("Sample size: {}".format(sample[0][0].size()))
torchvision.transforms.ToPILImage()(sample[0])

Sample class: 5
Sample size: torch.Size([28, 28])


In [5]:
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=32)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=32)
# loader speedup could be achieved by following parameters
# test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=1024,
#                                           pin_memory=True, num_workers=4, shuffle=False)

# Model definition with sequential nn.Module (easy way)

In [6]:
class NnModuleSequential(nn.Module):
    def __init__(self, img_size):
        super(NnModuleSequential,self).__init__()
        
        self.conv1 = nn.Sequential(
                nn.Conv2d(1,32, kernel_size=5),
                nn.ReLU(),
                nn.BatchNorm2d(32),
                nn.Conv2d(32, 64, kernel_size=5),
                nn.ReLU(),
                nn.BatchNorm2d(64),
                nn.MaxPool2d(2,2),
        )
        
        self.fc1 = nn.Linear((img_size-4*2)*(img_size-4*2)*64//4, 1024)
        self.fc2 = nn.Linear(1024, 10)

    def forward(self, x):
        out = self.conv1(x)
        out = out.view(x.size(0), -1)
        
        out = self.fc1(out)
        out = self.fc2(out)
        return out

### Testing that the initial model works

In [7]:
model = NnModuleSequential(img_size=28).cuda()

In [8]:
x = Variable(mnist_train[0][0].unsqueeze(0))
x = x.cuda()
model(x)

Variable containing:
-0.2653  0.4983  0.8471 -0.2105 -0.1330 -0.0413 -0.3417  0.3848 -0.1016  0.0332
[torch.cuda.FloatTensor of size 1x10 (GPU 0)]

### Accuracy testing

In [9]:
def test_model_accuracy(model, data_loader, use_cuda=False):
    model.eval() # Set model in evaluation mode so dropout etc won't affect
    test_loss, correct = 0, 0
    
    for data, target in data_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        if use_cuda:
            data, target = data.cuda(), target.cuda()
            
        output = model(data)
        test_loss += F.cross_entropy(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(data_loader.dataset)
    print('Test set: Average loss: {:.6f}, Accuracy: {}/{} ({:.2f}%)'.format(
                test_loss, correct, len(data_loader.dataset),
                100. * correct / len(data_loader.dataset)))

In [10]:
# We now have model with random accuracy
test_model_accuracy(model, test_loader, use_cuda=True)

Test set: Average loss: 2.303807, Accuracy: 748/10000 (7.48%)


In [11]:
%%timeit -r 3
test_model_accuracy(model, test_loader, use_cuda=True)

Test set: Average loss: 2.303807, Accuracy: 748/10000 (7.48%)
Test set: Average loss: 2.303807, Accuracy: 748/10000 (7.48%)
Test set: Average loss: 2.303807, Accuracy: 748/10000 (7.48%)
Test set: Average loss: 2.303807, Accuracy: 748/10000 (7.48%)
839 ms ± 17 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### Training the model

In [12]:
def train_epoch(model, data_loader, optimizer, log_interval_percentage=.35, use_cuda=False):
    model.train()
    train_loss = []
    log_interval = int((len(train_loader.dataset)/train_loader.batch_size) * log_interval_percentage)
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data = Variable(data)
        target = Variable(target)
        if use_cuda:
            data, target = data.cuda(), target.cuda()

        predict = model(data)
        loss = F.cross_entropy(predict, target)
        train_loss.append(loss.data[0])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_idx % log_interval == 0) and (batch_idx > 0):
            print('Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                        100. * batch_idx / len(train_loader), np.mean(train_loss)))
    print(("Epoch: {}, Loss: {:.6f}").format(epoch , np.mean(train_loss)))

In [13]:
total_epochs = 0

In [14]:
%%time
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
for epoch in range(6):
    total_epochs += 1
    train_epoch(model, train_loader, optimizer, use_cuda=True)
    test_model_accuracy(model, test_loader, use_cuda=True)
print()

Epoch: 0 [20992/60000 (35%)]	Loss: 0.380523
Epoch: 0 [41984/60000 (70%)]	Loss: 0.292143
Epoch: 0, Loss: 0.257314
Test set: Average loss: 0.193515, Accuracy: 9622/10000 (96.22%)
Epoch: 1 [20992/60000 (35%)]	Loss: 0.104836
Epoch: 1 [41984/60000 (70%)]	Loss: 0.097014
Epoch: 1, Loss: 0.092383
Test set: Average loss: 0.113898, Accuracy: 9722/10000 (97.22%)
Epoch: 2 [20992/60000 (35%)]	Loss: 0.054125
Epoch: 2 [41984/60000 (70%)]	Loss: 0.057371
Epoch: 2, Loss: 0.054409
Test set: Average loss: 0.057635, Accuracy: 9849/10000 (98.49%)
Epoch: 3 [20992/60000 (35%)]	Loss: 0.052725
Epoch: 3 [41984/60000 (70%)]	Loss: 0.048415
Epoch: 3, Loss: 0.046256
Test set: Average loss: 0.063283, Accuracy: 9837/10000 (98.37%)
Epoch: 4 [20992/60000 (35%)]	Loss: 0.030338
Epoch: 4 [41984/60000 (70%)]	Loss: 0.032884
Epoch: 4, Loss: 0.032046
Test set: Average loss: 0.054931, Accuracy: 9856/10000 (98.56%)
Epoch: 5 [20992/60000 (35%)]	Loss: 0.020591
Epoch: 5 [41984/60000 (70%)]	Loss: 0.024754
Epoch: 5, Loss: 0.026569
Te

# Save & Load & Predict

### Saving

In [15]:
model_dir = './data/pytorch_model'
import os
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [16]:
saved_model_path = model_dir + '/full_model_' + str(total_epochs)
saved_model_path

'./data/pytorch_model/full_model_6'

In [17]:
torch.save(model.state_dict(), saved_model_path)

### Loading

In [18]:
loaded_model = NnModuleSequential(img_size=28)
loaded_model.load_state_dict(torch.load(saved_model_path))
loaded_model = loaded_model.cuda()

### Predicting

In [22]:
sample_idx = int(random.random()*100)
sample = mnist_test[sample_idx][0]
x = Variable(sample.unsqueeze(0))
x = x.cuda()

result = loaded_model(x)
print("Predicted class: {}".format(np.argmax(result.data.cpu().numpy()[0])))
torchvision.transforms.ToPILImage()(sample)

Predicted class: 3
